# Temperature on Electricity Demand (New England)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings

%matplotlib inline

# test number 3 

#For warning surpression
warnings.filterwarnings("ignore", category=FutureWarning)
#To reactivate warning
# warnings.filterwarnings("default", category=FutureWarning)

: 

## Background
Rough skeleton

1. Talk about electcity market 
2. Talk about heating and how its fractured
3. Talk about Schaffer paper and exisitng literature 
4. Motivations into what why we are looking into this and what the plan is 

## Electricity Demand (System Load)

https://www.iso-ne.com/isoexpress/

In [ ]:
states = ['ISONE CA', 'ME', 'NH', 'VT', 'CT', 'RI', 'SEMASS' ,'WCMASS', 'NEMASSBOST']

state_dfs = [pd.DataFrame(columns=['Date', 'Hour', 'Demand', 'Temperature']) for _ in states]

: 

#### DO NOT TOUCH (Cleaning for final project)

In [ ]:
# for year in range(2011, 2018):
#     path = f"data/demand/{year}_smd_hourly.xls"
#     if year >= 2016:
#         for i, state in enumerate(states):
#             state_df = pd.read_excel(path, sheet_name = state)
#             state_df = state_df[['Date', 'Hr_End', 'RT_Demand', 'Dry_Bulb']]
#             state_df = state_df.rename(columns = {'Hr_End' : 'Hour', 'RT_Demand' : 'Demand', 'Dry_Bulb' : 'Temperature'})
#             state_df['Date'] = pd.to_datetime(state_df['Date'], format = 'dd-bb-yy')
            
#             state_dfs[i] = pd.concat([state_dfs[i], state_df], ignore_index = True)
#     else:
#         for i, state in enumerate(states):
#             state_df = pd.read_excel(path, sheet_name = state)
#             state_df = state_df[['Date', 'Hour', 'DEMAND', 'DryBulb']]
#             state_df = state_df.rename(columns = {'DEMAND' : 'Demand', 'DryBulb' : 'Temperature'})
#             state_df['Date'] = pd.to_datetime(state_df['Date'], format = 'dd/mm/yy')
            
#             state_dfs[i] = pd.concat([state_dfs[i], state_df], ignore_index = True)
            
# for i in range(len(state_dfs)):
#     #Set index as datetime
#     state_dfs[i]['Datetime'] = pd.to_datetime(state_dfs[i]['Date']) + pd.to_timedelta(state_dfs[i]['Hour'], unit='h')
#     state_dfs[i] = state_dfs[i].set_index('Datetime')
    
#     #Add Year column
#     state_dfs[i]['Year'] = pd.to_datetime(state_dfs[i]['Date']).dt.year
#     columns = ['Year'] + [col for col in state_dfs[i].columns if col != 'Year']
#     state_dfs[i] = state_dfs[i][columns]
    
#     #Remove all rows with 0 demand
#     state_dfs[i] = state_dfs[i][state_dfs[i]['Demand'] != 0]

: 

#### Continue

In [ ]:
state_names = ['isone_ca', 'me', 'nh', 'vt', 'ct', 'ri', 'sema', 'wcma', 'nema']

state_names_full = ['ISO-NE Control Area', 'Maine', 'New Hampshire', 'Vermont',
                    'Connecticut', 'Rhode Island', 'SouthEastern Massachusetts',
                    'Western & Central Massachusetts', 'NorthEastern Massachusetts & Boston']

: 

In [ ]:
for i in range(len(state_names)):
    path = f"data/Others/demand/{state_names[i]}.csv"
    state_dfs[i] = pd.read_csv(path)
    state_dfs[i]['Datetime'] = pd.to_datetime(state_dfs[i]['Date']) + pd.to_timedelta(state_dfs[i]['Hour'], unit='h')
    state_dfs[i] = state_dfs[i].set_index('Datetime')
    state_dfs[i]["Date"] = pd.to_datetime(state_dfs[i]["Date"])

: 

In [ ]:
state_dfs[0]

: 

In [ ]:
state_dfs[0].plot.scatter('Temperature', 'Demand')
plt.title('Electricity Demand on Temperature (Entire ISO-NE)')

: 

In [ ]:
# Assuming `state_dfs` is a list of 8 DataFrames and `state_names` is a list of 8 titles
fig, axes = plt.subplots(2, 4, figsize=(15, 8))  # Creates a 2x4 grid of subplots

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each DataFrame and corresponding title
for i, (df, name) in enumerate(zip(state_dfs[1:], state_names_full[1:])):
    axes[i].scatter(df['Temperature'], df['Demand'])
    axes[i].set_title(name)
    axes[i].set_ylabel('Demand')
    axes[i].set_xlabel('Temperature')

fig.suptitle('Electricity Demand on Temperature', fontsize=20, fontweight='bold')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

: 

### Natural Gas Prices

https://www.eia.gov/dnav/ng/ng_pri_sum_a_EPG0_PG1_DMcf_m.htm

We control for natural gas prices because 

In [ ]:
ng = pd.read_excel('data/NG_PRI_SUM_A_EPG0_PG1_DMCF_M.xls', sheet_name = 'Data 1',
                   skiprows = 2)

ng = ng[['Date', 'Natural Gas Citygate Price in Connecticut (Dollars per Thousand Cubic Feet)', 
         'Natural Gas Citygate Price in Maine (Dollars per Thousand Cubic Feet)', 
         'Natural Gas Citygate Price in Massachusetts (Dollars per Thousand Cubic Feet)', 
         'Natural Gas Citygate Price in New Hampshire (Dollars per Thousand Cubic Feet)',
         'Natural Gas Citygate Price in Rhode Island (Dollars per Thousand Cubic Feet)',
         'Natural Gas Citygate Price in Vermont (Dollars per Thousand Cubic Feet)']]

ng.columns = ['Date', 'Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont']

ng = ng[(ng['Date'] >= '2011-01-01') & (ng['Date'] <= '2017-12-31')]

ng['year_month'] = ng['Date'].dt.to_period('M')

ng

: 

In [ ]:
for i in range(len(state_dfs)):
    state_dfs[i]['year_month'] = state_dfs[i]['Date'].dt.to_period('M')
    #All New England - no NG price data
    if i == 0:
        state_dfs[i].drop(columns = ['year_month'], inplace = True)
        continue
    #Maine
    elif i == 1:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'Maine']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'Maine': 'Natural Gas Price'})
    #New Hampsphire
    elif i == 2:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'New Hampshire']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'New Hampshire': 'Natural Gas Price'})
    #Vermont
    elif i == 3:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'Vermont']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'Vermont': 'Natural Gas Price'})
    #Connecticut
    elif i == 4:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'Connecticut']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'Connecticut': 'Natural Gas Price'})
    #Rhode Island
    elif i == 5:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'Rhode Island']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'Rhode Island': 'Natural Gas Price'})
    #Massachusetts
    else:
        state_dfs[i] = pd.merge(state_dfs[i], ng[['year_month', 'Massachusetts']], on = 'year_month', how = 'left')
        state_dfs[i] = state_dfs[i].rename(columns={'Massachusetts': 'Natural Gas Price'})
    state_dfs[i].drop(columns = ['year_month'], inplace = True)

: 

In [ ]:
state_dfs[1]

: 

### Generation Mix

https://www.iso-ne.com/isoexpress/web/reports/operations/-/tree/daily-gen-fuel-type

: 

: 

: 

: 

### Elasticity

In [ ]:
pct_change_dfs = []

for i in range(len(state_dfs)):
    pct_change_df = state_dfs[i].copy()
    
    # Calculate percentage changes for Demand and Temperature
    pct_change_df['Demand_pct_change'] = pct_change_df['Demand'].pct_change()
    pct_change_df['Temperature_pct_change'] = pct_change_df['Temperature'].pct_change()

    # Calculate Elasticity
    pct_change_df['Elasticity'] = pct_change_df['Demand_pct_change'] / pct_change_df['Temperature_pct_change']

    # Remove rows with NaN or infinite Elasticity values
    pct_change_df = pct_change_df.dropna(subset=['Elasticity'])
    # pct_change_df = pct_change_df[~pct_change_df['Elasticity'].isin([np.inf, -np.inf])]

    # Append the modified DataFrame to pct_change_dfs
    pct_change_dfs.append(pct_change_df)

: 

In [ ]:
pct_change_dfs[1]

: 

In [ ]:
pct_change_dfs[0].plot.scatter('Temperature', 'Elasticity')
plt.title('Demand Elasticity on Temperature (New England)')

: 

In [ ]:
# Assuming `state_dfs` is a list of 8 DataFrames and `state_names` is a list of 8 titles
fig, axes = plt.subplots(2, 4, figsize=(15, 8))  # Creates a 2x4 grid of subplots

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each DataFrame and corresponding title
for i, (df, name) in enumerate(zip(pct_change_dfs[1:], state_names_full[1:])):
    axes[i].scatter(df['Temperature'], df['Elasticity'])
    axes[i].set_title(name)
    axes[i].set_ylabel('Elasticity')
    axes[i].set_xlabel('Temperature')

fig.suptitle('Demand Elasticity on Temperature', fontsize=20, fontweight='bold')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

: 

: 